In [1]:
import xarray as xr
from cdo import Cdo
cdo = Cdo()
import pandas as pd

In [27]:
import sys
sys.path.append("/home/abhi/Documents/mygit/postBC_diagnostic/src/scripts")
from recipes import *

In [28]:
data_dir = '/home/abhi/Documents/data/OBSERVATION/IMD'

In [47]:
!ls {data_dir}/tmin

1971-2000_ll100.nc	     1971-2000_rm_ll25.nc
1971-2000_rm_ll25_360day.nc  IMD_MinT_1969_2005.nc


In [96]:
files = {
    'pr': f'{data_dir}/precip/IMD_pr_1901-2017.nc',
    'tas': f'{data_dir}/tmean/IMD_MeanT_1969-2016.nc',
    'tasmax': f'{data_dir}/tmax/IMD_MaxT_1969-2016.nc',
    'tasmin': f'{data_dir}/tmin/IMD_MinT_1969-2016.nc'
}

In [97]:
def extract_ll_ds(nc, lon=77.75, lat=20.93):
    o = cdo.remapnn(f'lon={lon}/lat={lat}',
               input=f'-setvar,value -monmean -selyear,1976/2016 {nc}')
    
    ds = xr.open_dataset(o).mean(['lon', 'lat']).resample(time='M').mean('time')
    
    return(ds)
    

In [98]:
varIndex = pd.Index(['pr', 'tas', 'tasmax', 'tasmin'], name='variable')

In [99]:
ds = xr.concat([extract_ll_ds(files[var]) for var in varIndex], 
              dim=varIndex)

In [100]:
df = ds.to_dataframe().reset_index()

In [101]:
def get_seas(month):
    if month in [12, 1, 2]:
        seas = 'DJF'
    elif month in [3, 4, 5]:
        seas = 'MAM'
    elif month in [6, 7, 8, 9]:
        seas = 'JJAS'
    elif month in [10, 11]:
        seas = 'ON'
        
    return(seas)

In [102]:
df['year'] = df.time.dt.year
df['month'] = df.time.dt.month

In [103]:
df['seas'] = df.month.apply(get_seas)

In [104]:
df.head()

,time,variable,value,year,month,seas
0,1976-01-31,pr,0.024217,1976,1,DJF
1,1976-01-31,tas,20.474291,1976,1,DJF
2,1976-01-31,tasmax,28.292179,1976,1,DJF
3,1976-01-31,tasmin,12.656405,1976,1,DJF
4,1976-02-29,pr,0.000000,1976,2,DJF


In [105]:
df_seas_ymean = df.groupby(['variable', 'seas', 'year']).mean().reset_index().drop('month', axis=1)

In [106]:
df_ann_ymean = df.groupby(['variable', 'year']).mean().reset_index().drop('month', axis=1)
df_ann_ymean['seas'] = 'Annual'

In [107]:
df_comb = pd.concat([df_seas_ymean, df_ann_ymean], sort=False)

In [109]:
def get_tslice(year):
    if year in range(1976, 2005 + 1):
        tslice = 'baseline'
    elif year in range(2005, 2016 + 1):
        tslice = '2005-2016'
        
    return(tslice)

In [110]:
df_comb['tslice'] = df.year.apply(get_tslice)

In [111]:
df_comb.head()

,variable,seas,year,value,tslice
0,pr,DJF,1976,0.008072,baseline
1,pr,DJF,1977,0.036188,baseline
2,pr,DJF,1978,0.935086,baseline
3,pr,DJF,1979,1.605736,baseline
4,pr,DJF,1980,0.579395,baseline


In [112]:
df_comb.to_csv('/home/abhi/Documents/mygit/NEX-Analysis/pickles/Amravati/IMD/Amravati_allvars_seas_ymean_1976-2016.csv',
              index=False)